In [1]:
import torch
import torch.autograd
from torch.autograd import Variable

def mlp(x, Weight, Bias, Func):
    f = Variable(x, requires_grad=False)
    NumOfLayers = len(Weight)
    for i in range(NumOfLayers):
        f = Func[i](torch.matmul(Weight[i], f) + Bias[i])
    return f

## Part 1

In [2]:
sizes = [1, 2, 1]

x = 10*torch.rand(sizes[0]).double()

Weight = []
Bias = []
Func = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)
    Func.append(torch.sigmoid)

f = mlp(x, Weight, Bias, Func)

print("Outputs are:")

for i in range(sizes[-1]):
    print(f[i][-1])

Outputs are:
tensor(0.7291, dtype=torch.float64, grad_fn=<SelectBackward>)


## Part 2

In [3]:
sizes = [4, 2, 1]

x = 10*torch.rand(sizes[0]).double()

Weight = []
Bias = []
Func = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)
    Func.append(torch.sigmoid)

f = mlp(x, Weight, Bias, Func)

print("Outputs are:")

for i in range(sizes[-1]):
    print(f[i][-1])

Outputs are:
tensor(0.3035, dtype=torch.float64, grad_fn=<SelectBackward>)


### İris Dataset

In [5]:
import pandas as pd
import numpy as np

def logsumexp(b):
    return torch.log(1 + torch.exp(b))

sizes = [4, 2, 1]

df_iris = pd.read_csv(u'data/iris.txt',sep=' ')

y = torch.from_numpy(np.array(df_iris['c'])).double()
y[y>1]=0

x = torch.from_numpy(np.array(df_iris[['sl','sw','pl','pw']])).double()

Weight = []
Bias = []
Func = []

for i in range(len(sizes)-1):
    # For layer i, Weights are a S_{i+1} \times S_{i} matrix
    W = Variable(20*torch.randn(sizes[i+1], sizes[i]).double(), requires_grad=True)
    # For layer i, Biases are a S_{i+1} \times 1 matrix (a vector)
    b = Variable(torch.randn(sizes[i+1], 1).double(), requires_grad=True)
    
    Weight.append(W)
    Bias.append(b)
    Func.append(torch.sigmoid)

eta = 0.0005

for epoch in range(1000):
    for i in range(len(y)):
        f = mlp(x[i,:], Weight, Bias, Func)[-1][-1]
        
        E = -y[i]*f + logsumexp(f)
        
        E.backward()
    
    
    for i in range(len(Weight)):
        Weight[i].data.add_(-eta * Weight[i].grad.data)
    
        Weight[i].grad.zero_()

result = torch.zeros(len(y))
        
for i in range(len(y)):
    result[i] = torch.round(mlp(x[i,:], Weight, Bias, Func)[-1][-1])

confusion_matrix = torch.zeros((2,2))

for i in range(len(result)):
    confusion_matrix[int(y[i] - 1), int(result[i] - 1)] += 1
    
accuracy = torch.sum(torch.diag(confusion_matrix))/torch.sum(confusion_matrix)

print("Accuracy:", accuracy*100, "%")

Accuracy: tensor(33.3333) %
